In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
emotions=pd.read_csv('archive/emotions.txt',sep=";",header=None)
emotions=emotions.rename(columns={0:'text',1:"sentiment"})
le=LabelEncoder()
emotions['encoded']=le.fit_transform(emotions['sentiment'])

In [6]:
coded=emotions
coded=coded.drop(columns=['sentiment'])
coded

,text,encoded
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0
...,...,...
19995,im having ssa examination tomorrow in the morn...,4
19996,i constantly worry about their fight against n...,2
19997,i feel its important to share this info for th...,2
19998,i truly feel that if you are passionate enough...,2


In [74]:
train = coded.sample(frac=0.8)
test=coded.drop(train.index)

In [75]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=False,num_labels=24,args={
                                                                                             'reprocess_input_data': True,
                                                                                             'overwrite_output_dir': True,
                                                                                             'fp16': False,
                                                                                             'do_lower_case': False,
                                                                                             'num_train_epochs': 2,
                                                                                             
                                                                                             'regression': False,
                                                                                      
                                                                                             "learning_rate":4e-5,
                                                                                             'weight_decay':0.0,
                                                                                             "save_eval_checkpoints": False,
                                                                                             "save_model_every_epoch": False,
                                                                                             "silent": False})

model.train_model(train)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/16000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2000 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2000 [00:00<?, ?it/s]

(4000, 0.32999951148059337)

In [76]:
from sklearn.metrics import classification_report, accuracy_score
test_result, test_model_outputs, test_wrong_predictions = model.eval_model(test,acc=accuracy_score)

C:\Users\nigel\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/500 [00:00<?, ?it/s]

In [77]:
print(test_result)

{'mcc': 0.9160214317746849, 'acc': 0.936, 'eval_loss': 0.1799497791969916}


In [78]:
pickle.dump(model,open('bert.pkl','wb'))

In [79]:
bert=pickle.load(open('bert.pkl','rb'))

In [80]:
preds=bert.predict(['i am mad'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [83]:
le.inverse_transform(preds[0])[0]

'anger'